In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import random
import cv2
import numpy as np
!pip install pyefd
import pyefd
from google.colab.patches import cv2_imshow

In [ ]:
# Env vars
torch.use_deterministic_algorithms(True)

# Const vars
ROOT_DIR = "./"
RAND_SEED = 0
DEVICE = "cpu"

NUM_CLASSES = 3
EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 500
NUM_TRAIN_BATCHES = 60000 // BATCH_SIZE
NUM_VAL_BATCHES = 10000 // BATCH_SIZE
LOSS_FN = nn.CrossEntropyLoss()
TRAIN_SPLIT = 0.8


# deterministic worker re-seeding
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

    # custom dataset for quickdraw classes circle, square, triangle
class QuickdrawDataset(Dataset):
  def __init__(self, imgs, num_circle, num_square, num_triangle, transform):
    self.imgs = imgs
    self.num_circle = num_circle
    self.num_square = num_square
    self.num_triangle = num_triangle
    self.len = num_circle + num_square + num_triangle
    self.transform = transform

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    img = self.imgs[idx, :, :]
    x = self.transform(img)
    if idx < self.num_circle:
      y = 0
    elif idx < self.num_circle + self.num_square:
      y = 1
    else:
      y = 2
    return x, y

Section 2: Original images through linear classifier (to compare)

In [ ]:
from torch.functional import Tensor
# mlp taking quickdraw images
class LinearClassifierOriginal(nn.Module):
    def __init__(self):
        super(LinearClassifierOriginal, self).__init__()
        self.flatten = nn.Flatten()
        self.mlp = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, NUM_CLASSES))

    def forward(self, x):
        x = self.flatten(x)
        out = self.mlp(x)
        return out

# Define transformation(s) to be applied to dataset-
transforms_norm = T.Compose(
      [
          T.ToTensor(), # scales integer inputs in the range [0, 255] into the range [0.0, 1.0]
          T.Normalize(mean=(0.138), std=(0.296)) # Quickdraw mean and stdev (35.213, 75.588), divided by 255
          
      ]
  )

# transform functions - take PIL image, return img as 1x28x28 torch tensor
def original_transform_train(img):
  # retrun normalized image
  return transforms_norm(img)

# add rotations and translations at test time
# return torch tensor
def original_transform_test(img):
    img = transforms_norm(img)
    angle=((random.random())*60)-30
    
    transY = random.randint(-3, 3)
    transX = random.randint(-3, 3)
    new_img = T.functional.affine(img,angle,[transX,transY],1,0) 
    return new_img


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # put the model in train mode
    total_loss = 0
    total_correct = 0
    # for each batch in the training set compute loss and update model parameters
    for batch, (X, y) in enumerate(dataloader):
    # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation to update model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print current training metrics for user
        loss_val = loss.item()
        if batch % 100 == 0:
            current = batch * len(X)
            print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")

        pred = pred.argmax(dim=1, keepdim=True)
        correct = pred.eq(y.view_as(pred)).sum().item()
        total_correct += correct
        total_loss += loss_val
        # print(f"train loss: {loss_val:>7f}   train accuracy: {correct / BATCH_SIZE:>7f}   [batch: {batch + 1:>3d}/{NUM_TRAIN_BATCHES:>3d}]")      
    print(f"\nepoch avg train loss: {total_loss / (size / BATCH_SIZE):>7f}   epoch avg train accuracy: {total_correct / size:>7f}")

def eval_loop(dataloader, model):
  model.eval()
  size = len(dataloader.dataset)
  with torch.no_grad():
    total_correct = 0
    for X, y in dataloader:
      X, y = X.to(DEVICE), y.to(DEVICE)

      out = model(X)
      pred = out.argmax(dim=1, keepdim=True)
      total_correct += pred.eq(y.view_as(pred)).sum().item()

    accuracy = total_correct / size
    print(f"test accuracy: {accuracy:>7f}")



In [ ]:
# download dataset and create train/test split
!gsutil -m cp gs://quickdraw_dataset/full/numpy_bitmap/circle.npy /circle.npy
!gsutil -m cp gs://quickdraw_dataset/full/numpy_bitmap/square.npy /square.npy
!gsutil -m cp gs://quickdraw_dataset/full/numpy_bitmap/triangle.npy /triangle.npy

circle_imgs = np.load("/circle.npy")
circle_len = circle_imgs.shape[0]
circle_train_num = int(circle_len * TRAIN_SPLIT)
circle_test_num = circle_len - circle_train_num
imgs_temp=[]

  #vector of means= np.mean(circle_imgs, axis = 1)
  #vector of std= np.std(circle_imgs, axis = 1); take mean of the stds

#calculate quickdraw mean and std
# quickdraw_mean = np.mean(np.mean(circle_imgs, axis = 1))
# quickdraw_std = np.mean(np.std(circle_imgs, axis = 1))

for i in range(circle_len):
  #2nd dimension is the unrolled image
  #mean of each, circle[a][ith], is the unrolled image 
  #returns a vector of means
  #take the mean of the vector
    imgs_temp.append(circle_imgs[i][:].reshape([28,28,1]))
imgs_temp=np.array(imgs_temp)
imgs_temp=imgs_temp.reshape(circle_len,28,28)
circle_train_imgs = imgs_temp[:circle_train_num, :, :]
circle_test_imgs = imgs_temp[circle_train_num:, :, :]


square_imgs = np.load("/square.npy")
square_len = square_imgs.shape[0]
square_train_num = int(square_len * TRAIN_SPLIT)
square_test_num = square_len - square_train_num
imgs_temp=[]
for i in range(square_len):
    imgs_temp.append(square_imgs[i][:].reshape([28,28,1]))
imgs_temp=np.array(imgs_temp)
imgs_temp=imgs_temp.reshape(square_len,28,28)
square_train_imgs = imgs_temp[:square_train_num, :, :]
square_test_imgs = imgs_temp[square_train_num:, :, :]

triangle_imgs = np.load("/triangle.npy")
triangle_len = triangle_imgs.shape[0]
triangle_train_num = int(triangle_len * TRAIN_SPLIT)
triangle_test_num = triangle_len - triangle_train_num
imgs_temp=[]
for i in range(triangle_len):
    imgs_temp.append(triangle_imgs[i][:].reshape([28,28,1]))
imgs_temp=np.array(imgs_temp)
imgs_temp=imgs_temp.reshape(triangle_len,28,28)
triangle_train_imgs = imgs_temp[:triangle_train_num, :, :]
triangle_test_imgs = imgs_temp[triangle_train_num:, :, :]

train_imgs = np.concatenate((circle_train_imgs, square_train_imgs, triangle_train_imgs), axis=0)
test_imgs = np.concatenate((circle_test_imgs, square_test_imgs, triangle_test_imgs), axis=0)

# weed out bad imgs train indexes: [34843, 140511, 206125]
circle_train_num -= 1
square_train_num -= 1
triangle_train_num -= 1
train_imgs = np.delete(train_imgs, [34843, 140511, 206125], axis=0)

Copying gs://quickdraw_dataset/full/numpy_bitmap/circle.npy...
\ [1/1 files][ 91.9 MiB/ 91.9 MiB] 100% Done                                    
Operation completed over 1 objects/91.9 MiB.                                     
Copying gs://quickdraw_dataset/full/numpy_bitmap/square.npy...
- [1/1 files][ 93.6 MiB/ 93.6 MiB] 100% Done                                    
Operation completed over 1 objects/93.6 MiB.                                     
Copying gs://quickdraw_dataset/full/numpy_bitmap/triangle.npy...
\ [1/1 files][ 92.1 MiB/ 92.1 MiB] 100% Done                                    
Operation completed over 1 objects/92.1 MiB.                                     


In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_original_data = QuickdrawDataset(train_imgs, circle_train_num, square_train_num, triangle_train_num, transform=original_transform_train)
eval_original_data = QuickdrawDataset(test_imgs, circle_test_num, square_test_num, triangle_test_num, transform=original_transform_train)
test_original_data = QuickdrawDataset(test_imgs, circle_test_num, square_test_num, triangle_test_num, transform=original_transform_test)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_original_loader = DataLoader(train_original_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
eval_original_loader = DataLoader(eval_original_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_original_loader = DataLoader(test_original_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = LinearClassifierOriginal()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


# train for EPOCHS number of epochs then evaluate on test data with affine transformations
for i in range(EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_original_loader,model=model,loss_fn=LOSS_FN,optimizer=optimizer)
    eval_loop(dataloader=eval_original_loader,model=model)
    print("\n-------------------------------\n")
eval_loop(dataloader=test_original_loader,model=model)

Epoch 1

loss: 1.096165  [    0/296949]
loss: 0.057595  [50000/296949]
loss: 0.052520  [100000/296949]
loss: 0.054838  [150000/296949]
loss: 0.068327  [200000/296949]
loss: 0.083692  [250000/296949]

epoch avg train loss: 0.084184   epoch avg train accuracy: 0.968207
test accuracy: 0.974892

-------------------------------

Epoch 2

loss: 0.054826  [    0/296949]
loss: 0.041243  [50000/296949]
loss: 0.071642  [100000/296949]
loss: 0.104768  [150000/296949]
loss: 0.048272  [200000/296949]
loss: 0.085840  [250000/296949]

epoch avg train loss: 0.057400   epoch avg train accuracy: 0.978070
test accuracy: 0.976737

-------------------------------

Epoch 3

loss: 0.033763  [    0/296949]
loss: 0.072470  [50000/296949]
loss: 0.034303  [100000/296949]
loss: 0.053285  [150000/296949]
loss: 0.062095  [200000/296949]
loss: 0.039147  [250000/296949]

epoch avg train loss: 0.049268   epoch avg train accuracy: 0.981182
test accuracy: 0.977155

-------------------------------

Epoch 4

loss: 0.04433

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_original_data = QuickdrawDataset(train_imgs, circle_train_num, square_train_num, triangle_train_num, transform=original_transform_test)
eval_original_data = QuickdrawDataset(test_imgs, circle_test_num, square_test_num, triangle_test_num, transform=original_transform_test)
test_original_data = QuickdrawDataset(test_imgs, circle_test_num, square_test_num, triangle_test_num, transform=original_transform_train)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_original_loader = DataLoader(train_original_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
eval_original_loader = DataLoader(eval_original_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_original_loader = DataLoader(test_original_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = LinearClassifierOriginal()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


# train for EPOCHS number of epochs then evaluate on test data with affine transformations
for i in range(EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_original_loader,model=model,loss_fn=LOSS_FN,optimizer=optimizer)
    eval_loop(dataloader=eval_original_loader,model=model)
    print("\n-------------------------------\n")
eval_loop(dataloader=test_original_loader,model=model)

Epoch 1

loss: 1.094395  [    0/296949]
loss: 0.178236  [50000/296949]
loss: 0.175739  [100000/296949]
loss: 0.137356  [150000/296949]
loss: 0.140548  [200000/296949]
loss: 0.132038  [250000/296949]

epoch avg train loss: 0.180883   epoch avg train accuracy: 0.927715
test accuracy: 0.954188

-------------------------------

Epoch 2

loss: 0.113573  [    0/296949]
loss: 0.083994  [50000/296949]
loss: 0.093136  [100000/296949]
loss: 0.144082  [150000/296949]
loss: 0.102903  [200000/296949]
loss: 0.115459  [250000/296949]

epoch avg train loss: 0.105040   epoch avg train accuracy: 0.957936
test accuracy: 0.961516

-------------------------------

Epoch 3

loss: 0.096904  [    0/296949]
loss: 0.101834  [50000/296949]
loss: 0.073112  [100000/296949]
loss: 0.077982  [150000/296949]
loss: 0.102032  [200000/296949]
loss: 0.082610  [250000/296949]

epoch avg train loss: 0.088717   epoch avg train accuracy: 0.964465
test accuracy: 0.965651

-------------------------------

Epoch 4

loss: 0.08277